# 아웃_노아웃으로 나눈 데이터로 학습

In [2]:
from sklearn.model_selection import train_test_split

In [3]:
from sklearn.multioutput import MultiOutputRegressor
from lightgbm import LGBMRegressor

In [5]:
import numpy as np
import pandas as pd
from sklearn.metrics import mean_squared_error, mean_absolute_error

def rmse(y_true, y_pred):
    return np.sqrt(mean_squared_error(y_true, y_pred))

def mse(y_true, y_pred):
    return mean_squared_error(y_true, y_pred)

def mae(y_true, y_pred):
    return mean_absolute_error(y_true, y_pred)

def mape(y_true, y_pred):
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100  # 백분율로 변환


In [25]:
data = pd.read_excel("mlb_dp_20.xlsx")
data.head()

KeyboardInterrupt: 

In [8]:
data.drop("Unnamed: 0", axis=1, inplace=True)

In [11]:
data.head()

,id,ball_type,pitch_type,velocity,exit_velocity,hit_dist,zone_num,pitcher_hand,batter_hand,ball,...,temp,wind,rain,theta_p,theta_n,distance,player_age,slg_percent,isolated_power,babip
0,676356,0,2,91.2,88.5,3,14,0,0,1,...,23.0,0.0,0,115.306158,-25.306158,91.654771,24,0.429,0.167,0.281
1,676356,1,2,86.3,105.0,228,9,0,0,0,...,21.3,11.6,0,103.751442,-13.751442,189.305002,24,0.429,0.167,0.281
2,676356,0,1,95.7,91.9,100,8,0,0,0,...,23.0,0.0,0,41.540762,48.459238,96.804764,24,0.429,0.167,0.281
3,676356,0,5,83.3,80.7,208,6,0,0,1,...,23.0,0.0,0,91.562115,-1.562115,168.446016,24,0.429,0.167,0.281
4,676356,0,2,91.2,88.5,3,14,0,0,1,...,23.0,0.0,0,116.408340,-26.408340,92.322138,24,0.429,0.167,0.281


In [13]:
# 아웃, 노아웃으로 나누는 함수
def ball_type_enc(type):
    if type == 0 or type == 5 or type == 6:
        return 0 # out
    else:
        return 1 # no_out

In [15]:
# 아웃 -> 0, 노아웃 -> 1로 인코딩
data["ball_type2"] = data["ball_type"].apply(lambda x: ball_type_enc(x))

In [17]:
data.head()

,id,ball_type,pitch_type,velocity,exit_velocity,hit_dist,zone_num,pitcher_hand,batter_hand,ball,...,wind,rain,theta_p,theta_n,distance,player_age,slg_percent,isolated_power,babip,ball_type2
0,676356,0,2,91.2,88.5,3,14,0,0,1,...,0.0,0,115.306158,-25.306158,91.654771,24,0.429,0.167,0.281,0
1,676356,1,2,86.3,105.0,228,9,0,0,0,...,11.6,0,103.751442,-13.751442,189.305002,24,0.429,0.167,0.281,1
2,676356,0,1,95.7,91.9,100,8,0,0,0,...,0.0,0,41.540762,48.459238,96.804764,24,0.429,0.167,0.281,0
3,676356,0,5,83.3,80.7,208,6,0,0,1,...,0.0,0,91.562115,-1.562115,168.446016,24,0.429,0.167,0.281,0
4,676356,0,2,91.2,88.5,3,14,0,0,1,...,0.0,0,116.408340,-26.408340,92.322138,24,0.429,0.167,0.281,0


In [19]:
X = data.drop(columns=["theta_p", "theta_n", "distance", "ball_type"])
y = data[["theta_p", "distance"]]

In [21]:
# 1차: Train(80%) + Test(20%) 분할
X_train_val, X_test, y_train_val, y_test = train_test_split(X, y, test_size=0.2, random_state=1234)

# 2차: Train(60%) + Validation(20%) 분할 (Train+Validation을 다시 나눔)
X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, test_size=0.25, random_state=1234)

In [23]:
lgbm_model = LGBMRegressor(n_jobs=-1, random_state=1234)
lgbm_model = MultiOutputRegressor(lgbm_model)
lgbm_model.fit(X_train, y_train)
print("lightGBM RMSE : ", rmse(y_val, lgbm_model.predict(X_val)))
print("lightGBM MAPE : ", mape(y_val, lgbm_model.predict(X_val)))

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.008541 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 2212
[LightGBM] [Info] Number of data points in the train set: 65982, number of used features: 19
[LightGBM] [Info] Start training from score 92.026926
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.008915 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 2212
[LightGBM] [Info] Number of data points in the train set: 65982, number of used features: 19
[LightGBM] [Info] Start training from score 166.914255
lightGBM RMSE :  22.99033910335381
lightGBM MAPE :  24.949058794014917
